In [1]:
#My Methods
#Also bring csv's
import numpy as np
import pandas as pd
import scipy.integrate
import sys
import json
import csv
import math
MIDSQUARES = []
LINEARCONGRUENTIAL = []
MIXEDCONGRUENTIAL = []
MULTIPLICATORCONGRUENTIAL = []
CONVINATEDCONGRUENTIAL = []

In [2]:
#Table getters
#Chi-Square table
ChiSquare = []
with open('./resources/ChiSquare.csv', 'r' ) as csvFile:
    reader = csv.reader(csvFile)
    for row in reader:
        ChiSquare.append(row)
csvFile.close()

#Kolmogorov table
Kolmogorov = []
with open('./resources/Kolmogorov.csv', 'r' ) as csvFile:
    reader = csv.reader(csvFile)
    for row in reader:
        Kolmogorov.append(row)
csvFile.close()

In [81]:
def getChiValue(N, Significance):
    i = 0
    j = 0
    P = None
    while(len(ChiSquare)>j):
        if(str(N) == ChiSquare[j][0]):
            P = j
        j+=1
    while(len(ChiSquare[0])>i):
        if(str(Significance) == ChiSquare[0][i]):
            return float(ChiSquare[P][i])
        i+=1
def getKolmogorovValue(N, significance):
    i = 0
    j = 0
    P = None
    if(int(N)>50):
        if(significance == .001):
            return ((1.94947)/np.sqrt(N))
        elif(significance == .01):
            return (1.62762/np.sqrt(N))
        elif(significance == .02):
            return (1.51743/np.sqrt(N))
        elif(significance == .05):
            return (1.35810/np.sqrt(N))
        elif(significance == .1):
            return (1.22385/np.sqrt(N))
        elif(significance == .15):
            return (1.13795/np.sqrt(N))
        elif(significance == .2):
            return (1.07275/np.sqrt(N))
    else:
        while(len(Kolmogorov)>j):
            if(str(N) == Kolmogorov[j][0]):
                P = j
            j+=1
        while(len(Kolmogorov[0])>i):
            if(str(significance) == Kolmogorov[0][i]):
                return float(Kolmogorov[P][i])
            i+=1

In [106]:
#Random Numbers
#Mid-Squares
def midSquares(x0, reps):
    seed = np.array([], dtype=int)
    generator = []
    randoms = np.array([], dtype=int)
    rands = np.array([], dtype=int)
    periodFound = False

    for i in range(0,reps):
        seed = np.append(seed, x0)
        sqr = format(int(np.power(x0, 2)), '08')

        generator.append(sqr[:2] + "|" + sqr[2:6] + "|" + sqr[6:])
        generated = int(sqr[2:6])

        randoms = np.append(randoms, generated)
        x0 = generated
        rands = np.append(rands, x0 / 10000)
        for j in np.isin(seed, x0):
            if j:
                periodFound = True
                break
        if periodFound:
            break
    MIDSQUARES = rands
    df = {'Seed': seed.tolist(),
         'Generator': generator,
         'Generated': randoms.tolist(),
         'Randoms': rands.tolist()}
    d = pd.DataFrame(df)
    return d

In [107]:
midSquares(31234, 5)

[0.5562 0.9358 0.5721 0.7298 0.2608]


,Seed,Generator,Generated,Randoms
0,31234,97|5562|756,5562,0.5562
1,5562,30|9358|44,9358,0.9358
2,9358,87|5721|64,5721,0.5721
3,5721,32|7298|41,7298,0.7298
4,7298,53|2608|04,2608,0.2608


In [110]:
#Linear Congruential
def linearCongruential(x0, a, c, m, reps, contin=False):
    seed = np.array([], dtype=float)
    randoms = np.array([], dtype=float)
    rands = np.array([], dtype=float)
    periodFound = False

    for i in range(0, reps):
        seed = np.append(seed, x0)
        rep = np.mod((a * x0 + c), m)
        randoms = np.append(randoms, rep)
        x0 = rep
        rands = np.append(rands, rep / m)
        if not contin:
            for j in np.isin(seed, x0):
                if j:
                    periodFound = True
                    break
        if periodFound:
            break

    df = {'Seed': seed.tolist(),
         'Generated': randoms.tolist(),
         'Randoms': rands.tolist()}
    LINEARCONGRUENTIAL = rands
    MIXEDCONGRUENTIAL = rands
    MULTIPLICADORCONGRUENTIAL = rands
    d= pd.DataFrame(df)
    return d

In [111]:
linearCongruential(4,6,7,8,3)

[0.875 0.125 0.625]


,Seed,Generated,Randoms
0,4.0,7.0,0.875
1,7.0,1.0,0.125
2,1.0,5.0,0.625


In [86]:
def HullDobell(a, c, m):
    if isRelativePrime(c, m):
        facs = PrimeFactors(m)
        for i in (np.mod((a-1), facs) == 0):
            if not(i):
                return False
        if np.mod(m, 4) == 0:
            if np.mod(a-1,4) == 0:
                return True
    return False

In [87]:
def isRelativePrime(a, b):
    return np.gcd(int(a), int(b)) == 1

In [88]:
def isPrime(a):
    if num < 1:
        return False
    elif num == 2:
        return True
    else:
        for i in range(2, num):
            if np.mod(num, i) == 0:
                return False
        return True 

In [89]:
def PrimeFactors(n):
    i = 2
    factors = []
    while i * i <= n:
        if np.mod(n, i):
            i += 1
        else:
            n //= i
            factors.append(i)
    if n > 1:
        factors.append(n)
    return np.array(factors)

In [90]:
def suggest(x0, a, c, m):
    a1=a
    c1=c
    m1=m
    if(np.mod(m1,4)!=0):
        m1 -= np.mod(m1,4)
    while not isRelativePrime(c1, m1):
        c1+=1
    step2 = False
    primeFactors = PrimeFactors(m1)
    while not step2:
        check = (np.mod((a1-1), primeFactors) == 0)
        if( (not False in check) and np.mod(a1-1,4) == 0 ):
            step2 = True                    
        else:
            a1+=1                       
    return {'seed': int(x0),
            'mult' : int(a1),
            'inc' : int(c1),
            'mod': int(m1)}

In [91]:
#Mixed Congruential
def mixedCongruential(x0, a, c, m, reps):    
    if HullDobell(a, c, m):
        return linearCongruential(x0, a, c, m, reps)
    else:        
        df = {'Error': "Does not meets Hull Dobell",
              'Suggested': suggest(x0, a, c, m)}
    d=pd.DataFrame(df)
    return d

In [92]:
mixedCongruential(5,5,9,4,6)

,Seed,Generated,Randoms
0,5.0,2.0,0.50
1,2.0,3.0,0.75
2,3.0,0.0,0.00
3,0.0,1.0,0.25
4,1.0,2.0,0.50


In [93]:
#Congruelcial Multiplicativo
def multCongruential(x0, a, m, reps, contin=False):    
    if m > a and m > x0:
        return linearCongruential(x0, a, 0, m, reps, contin=contin)

In [94]:
multCongruential(5,3,7,2)

,Seed,Generated,Randoms
0,5.0,1.0,0.142857
1,1.0,3.0,0.428571


In [112]:
#Linear Congruential 1
def linearCongruential1(x0, a, c, m, reps, contin=False):
    seed = np.array([], dtype=float)
    randoms = np.array([], dtype=float)
    rands = np.array([], dtype=float)
    periodFound = False

    for i in range(0, reps):
        seed = np.append(seed, x0)
        rep = np.mod((a * x0 + c), m)
        randoms = np.append(randoms, rep)
        x0 = rep
        rands = np.append(rands, rep / m)
        if not contin:
            for j in np.isin(seed, x0):
                if j:
                    periodFound = True
                    break
        if periodFound:
            break

    df = {'Seed': seed.tolist(),
         'Generated': randoms.tolist(),
         'Randoms': rands.tolist()}
    CONVINATEDCONGRUENTIAL = rands
    return df
#Congruelcial Multiplicativo 1
def multCongruential1(x0, a, m, reps, contin=False):    
    if m > a and m > x0:
        return linearCongruential1(x0, a, 0, m, reps, contin=contin)

In [113]:
def combCongruential(x0, a, m, k, reps):
    lc = []
    gen = []
    rands = []
    mk = m[0]
    for i in range(0, k):
        lc.append(multCongruential1(x0[i], a[i], m[i], reps, True)["Randoms"])
    for i in range(0, reps):
        res = 0
        for j in range(0, k):
            res += np.mod((np.power(-1, j) * lc[j][i]), mk -1)
        gen.append(res)
        if res > 0 :
            rands.append(res / mk)
        elif res == 0:
            rands.append((mk-1)/mk)
            
    df = {'Seed': [],
          'Generated': gen,
          'Randoms': rands}
    
    cont = 1
    for rands in lc:
        df["Seed"].append(rands)
        cont+=1
    df['Seed'][0]
    df1 ={
        'Seed 1':df['Seed'][0],
        'Seed 2':df['Seed'][1],
        'Generated':gen,
        'Randoms': rands
    }
    d = pd.DataFrame(df1)
    return d

In [114]:
xinitial = [3,2]
astuff = [2,1]
mod=[4,5]
k = 2
combCongruential(xinitial,astuff,mod,k,5)

,Seed 1,Seed 2,Generated,Randoms
0,0.5,0.4,3.1,0.4
1,0.0,0.4,2.6,0.4
2,0.0,0.4,2.6,0.4
3,0.0,0.4,2.6,0.4
4,0.0,0.4,2.6,0.4


In [161]:
#Pruebas de Bondad
#Chi-Square
def chiSquareGoodnessTest(dataset, acceptance):
    function = lambda x : 1 / 1-0 if 0<=x and x<=1 else 0
    var = 1 #?
    minimum = np.amin(dataset)
    maximum = np.amax(dataset)
    n = len(dataset)
    rang = maximum - minimum
    k = 1+3.322*np.log10(n)
    size = rang / k    
    classmin = np.array([0], dtype=float)#init tablas
    classmax = np.array([0], dtype=float)
    foabsolute = np.array([0] ,dtype=float)
    oirelative = np.array([0], dtype=float)
    eitheory = np.array([0], dtype=float)
    chi2 = np.array([0], dtype=float) #fin init tablas   
    i = minimum    
    while i < maximum:
        classmin = np.append(classmin, i)
        classmax = np.append(classmax, i+size)
        freq = sum(1 for x in dataset if x >= i and x < (i+size))
        foabsolute = np.append(foabsolute, freq)
        oirelative = np.append(oirelative, freq / n)
        ei = scipy.integrate.quad(function, i, i+size)[0]
        eitheory = np.append(eitheory, ei)
        chi2 = np.append(chi2, np.power((freq / n - ei), 2) / ei)        
        i += size  
        
    sumChi2 = np.sum(chi2)
    chiTheory = getChiValue(var, acceptance)
    df1 = {
        "classmin":classmin,
        "classmax" : classmax,
        'foabsolute':foabsolute,
        'oirelative': oirelative,
        'eitheory': eitheory,
        'chi2': chi2,
        "Chi ^2 calculated" : sumChi2,
        "Chi ^2 Theoretical" : chiTheory,
        "Accepted" : "True" if sumChi2<=chiTheory else "False"
    }
    
    df = {
        "Chi ^2 calculated" : sumChi2,
        "Chi ^2 Theoretical" : chiTheory,
        "Accepted" : "True" if sumChi2<=chiTheory else "False"
    }
    d = pd.DataFrame(df1)
    return d

In [162]:
def kolmogorovGoodnessTest(dataset, acceptance):
    dataset = np.array(dataset)
    dataset.sort(axis=0)
    N = len(dataset)
    cumulativeFunction = lambda x : x / N
    cont = 1
    r = np.append([0], dataset)
    s = np.array([0], dtype=float)
    dp = np.array([0], dtype=float)
    dm = np.array([0], dtype=float)
    for i in dataset:
        s = np.append(s, cumulativeFunction(cont))
        dp = np.append(dp, np.abs(s[cont]-i))
        if cont == 1:
            dm = np.append(dm, np.abs(i))
        else:
            dm = np.append(dm, np.abs(i-(s[cont-1])))
        cont+=1;
        
    maxD = np.amax(np.append(np.amax(dp), np.amax(dm)))
    theoryD = getKolmogorovValue(N, acceptance)
    
    df = {
        "Maximum Distance": maxD,
        "Theory Distance" : theoryD,
        "Accepted" : "True" if maxD <= theoryD else "False"
    }
    return df

In [163]:
function = lambda x : 1 / 1-0 if 0<=x and x<=1 else 0
i = .0233
size = 1.17
ei = scipy.integrate.quad(function, i, i+size)[0]
print(ei)

0.9767000014785394


In [164]:
a =[0.018,
0.037,
0.156,
0.191,
0.213,
0.233,
0.281,
0.383,
0.392,
0.408,
0.411,
0.434,
0.469,
0.541,
0.553,
0.575,
0.598,
0.668,
0.671,
0.711,
0.719,
0.73,
0.77,
0.791,
0.819,
0.826,
0.894,
0.914,
0.984,
0.995]

In [165]:
acceptance = .05
chiSquareGoodnessTest(a,.05)

,classmin,classmax,foabsolute,oirelative,eitheory,chi2,Chi ^2 calculated,Chi ^2 Theoretical,Accepted
0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.077489,3.841,True
1,0.018000,0.183397,3.0,0.100000,0.165397,0.025858,0.077489,3.841,True
2,0.183397,0.348794,4.0,0.133333,0.165397,0.006216,0.077489,3.841,True
3,0.348794,0.514191,6.0,0.200000,0.165397,0.007239,0.077489,3.841,True
4,0.514191,0.679588,6.0,0.200000,0.165397,0.007239,0.077489,3.841,True
5,0.679588,0.844985,7.0,0.233333,0.165397,0.027905,0.077489,3.841,True
6,0.844985,1.010382,4.0,0.133333,0.155015,0.003032,0.077489,3.841,True
